In [2]:
ev = evaluation.Evaluation(ml_small)

Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Processing batch done
Process batch
Process batch
Processing batch done
Processing batch done
Process batch
Process batch
Processing batch done
Processing batch done
Process batch
Process batch
Processing batch done
Processing batch done


In [8]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
from models.ConstraintAutoRec import ConstraintAutoRec 
import models.NeuralLogicRec
import tensorflow as tf
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)
import itertools

ml_small = utils.common.ml_small



In [9]:
nlr = models.NeuralLogicRec.NLR(ml_small['user'], ml_small['dimensions'], epochs=1, embedding_dim=16, batch_size=20)

In [4]:
nlr.epochs = 5
for i in range(3):
    nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])
    print(ev.evaluate(nlr))

Epoch #0 Loss at step 30: 0.1612, time: 4.126
Epoch #1 Loss at step 30: 0.1529, time: 3.426
Epoch #2 Loss at step 30: 0.1396, time: 3.326
Epoch #3 Loss at step 30: 0.1290, time: 3.341
Epoch #4 Loss at step 30: 0.1193, time: 3.333
Batch nr 1 predicted
Batch nr 2 predicted
waiting for queue
processing results
Evaluating NeuralLogicRec |============================================================>| 100.0% 
{'accuracy': 0.43669181034482757, 'precision': 0.7975158542534428, 'recall': 0.4115342196859317, 'map@1': 0.5, 'map@5': 0.16666666666666666, 'map@10': 0.08338758680555555, 'diversity@5': 0.3440100524942826, 'diversity@10': 0.3515551350557251, 'epc@5': 0.9016734403773413, 'epc@10': 0.9219850066296027, 'epd@5': 0.3606010562486245, 'name': 'NeuralLogicRec', 'latent_dim': 128, 'epochs': 5, 'batch_size': 20}
Epoch #0 Loss at step 30: 0.1177, time: 3.318
Epoch #1 Loss at step 30: 0.1083, time: 3.360
Epoch #2 Loss at step 30: 0.1105, time: 3.299
Epoch #3 Loss at step 30: 0.1029, time: 3.268
Ep

In [10]:
nlr.epochs = 5
for i in range(3):
    nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])
    print(ev.evaluate(nlr))

W0725 15:18:16.759938 140578652948288 deprecation.py:323] From /home/markush/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1256: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch #0 Loss at step 30: 0.1587, time: 10.427
Epoch #1 Loss at step 30: 0.1250, time: 8.908
Epoch #2 Loss at step 30: 0.0996, time: 8.984
Epoch #3 Loss at step 30: 0.0909, time: 9.054
Epoch #4 Loss at step 30: 0.0840, time: 8.819
Batch nr 1 predicted
Batch nr 2 predicted
waiting for queue
processing results
Evaluating NeuralLogicRec |============================================================>| 100.0% 
{'accuracy': 0.43637556221889057, 'precision': 0.7923198233717079, 'recall': 0.42652430386919216, 'map@1': 0.5, 'map@5': 0.2007421875, 'map@10': 0.10066406250000001, 'diversity@5': 0.3453931110091504, 'diversity@10': 0.3512463610830536, 'epc@5': 0.9262681588170882, 'epc@10': 0.9108492862577728, 'epd@5': 0.3602412945398409, 'name': 'NeuralLogicRec', 'latent_dim': 128, 'epochs': 5, 'batch_size': 20}
Epoch #0 Loss at step 30: 0.1042, time: 8.931
Epoch #1 Loss at step 30: 0.0998, time: 8.757
Epoch #2 Loss at step 30: 0.0985, time: 8.782
Epoch #3 Loss at step 30: 0.0950, time: 8.883
Epoch #

In [6]:
# print(np.flip(np.argsort(nlr.predict_single_user(3))))
pred = nlr.model([4])
# inf = models.NeuralLogicRec.map_inference(nlr.model, pred)
print(np.sort(pred['likes']))
# print(np.sort(inf['likes'].numpy()))
# nlr.model(np.array(1), np.array(1))
# ev.evaluate(nlr)
# pred = nlr.model(tf.convert_to_tensor([1, 2, 3], dtype=tf.int32))

[[3.0696392e-06 3.9041042e-06 5.6028366e-06 ... 9.8202431e-01
  9.8493266e-01 9.8527825e-01]]


In [7]:
u = tf.convert_to_tensor([x for x in itertools.permutations(range(600), 2)])
u1 = u[:,0]
u2 = u[:,1]

In [86]:
num_movies = 10
num_user = 3
rated = tf.convert_to_tensor(np.arange(num_movies * num_user).reshape([num_user, num_movies]) % 3 == 0)
seen = tf.convert_to_tensor(np.arange(num_movies * num_user, num_movies * num_user * 2).reshape([num_user, num_movies]))
users = tf.convert_to_tensor(np.arange(num_user))
users = tf.convert_to_tensor([20, 10, 5, 0, 40])
print(rated)
print(seen)

tf.Tensor(
[[ True False False  True False False  True False False  True]
 [False False  True False False  True False False  True False]
 [False  True False False  True False False  True False False]], shape=(3, 10), dtype=bool)
tf.Tensor(
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]], shape=(3, 10), dtype=int64)


In [87]:
user_cross = tf.convert_to_tensor([x for x in itertools.permutations(users.numpy(), 2)])
print(user_cross)
rated_cross = tf.convert_to_tensor([tf.stack([rated[x[0]], rated[x[1]]], axis=0) for x in itertools.permutations(range(len(users.numpy())), 2)])
# print(rated_cross)

tf.Tensor(
[[20 10]
 [20  5]
 [20  0]
 [20 40]
 [10 20]
 [10  5]
 [10  0]
 [10 40]
 [ 5 20]
 [ 5 10]
 [ 5  0]
 [ 5 40]
 [ 0 20]
 [ 0 10]
 [ 0  5]
 [ 0 40]
 [40 20]
 [40 10]
 [40  5]
 [40  0]], shape=(20, 2), dtype=int32)


InvalidArgumentError: slice index 3 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [88]:
len(user_cross)

20

In [89]:
user_emb = tf.expand_dims(tf.nn.embedding_lookup(nlr.model.user_embedding, user_cross), axis=2)
print(user_emb[:, 0])
user_embedd = tf.tile(user_emb, [1, 1, num_movies, 1])
expanded_embedd = tf.reshape(nlr.model.item_embedding, [1, 1, num_movies, 24])
item_embedd = tf.tile(expanded_embedd, [len(user_cross), 2, 1, 1])
print(item_embed.shape)
print(user_embedd.shape)
inp = tf.concat([user_embedd, item_embedd], axis=-1)

tf.Tensor(
[[[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   0.08901728
   -0.90769035 -1.6418009  -0.3131709 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   

InvalidArgumentError: Input to reshape is a tensor with 83048 values, but the requested shape has 240 [Op:Reshape]

In [90]:
result = tf.squeeze(nlr.model.likes_estimator(inp))

NameError: name 'inp' is not defined

In [84]:
num_movies = 5
users = tf.convert_to_tensor([1, 10, 100])
embed_user = tf.expand_dims(users, axis=1)
users_1 = tf.tile(tf.expand_dims(embed_user, axis=1), [1, len(users), 1])
users_2 = tf.tile(tf.expand_dims(embed_user, axis=0), [len(users),1, 1])
# print(embed_user)
# embed_user = tf.tile(embed_user, [1, num_movies, 1])
# expanded_embedd = tf.expand_dims(nlr.model.item_embedding[0:num_movies], axis=0)
# embed_item = tf.tile(expanded_embedd, [len(users), 1, 1])
sim_input = tf.reduce_sum(tf.concat([users_1, users_2], axis=-1), axis=-1)
print(sim_input)

tf.Tensor(
[[  2  11 101]
 [ 11  20 110]
 [101 110 200]], shape=(3, 3), dtype=int32)


In [99]:
multi_ratings = tf.tile(tf.expand_dims(rated, axis=0), [3, 1, 1])

In [17]:
a = tf.expand_dims(tf.convert_to_tensor(10), axis=0)
b = tf.expand_dims(tf.convert_to_tensor(20), axis=0)
c = tf.convert_to_tensor([1,2,3,4])

In [18]:
tf.concat([a,b, c], axis=0)

<tf.Tensor: id=134207, shape=(6,), dtype=int32, numpy=array([10, 20,  1,  2,  3,  4], dtype=int32)>